In [2]:
from torch.nn import MSELoss, CrossEntropyLoss

from helpers.datasets.make_and_save.aggregated_signal import Aggregated_Signal_Dataframe_Handler
from helpers.datasets.constants import Names_of_Levels, Names_of_q_Squared_Vetos, Raw_Signal_Trial_Ranges, Numbers_of_Events_per_Set, Names_of_Splits
from helpers.experiment.experiment import CNN_Group, Deep_Sets_Group, Event_by_Event_Group
from helpers.experiment.results_table import Results_Table
from helpers.experiment.constants import Paths_to_Directories
from helpers.models.hardware_util import select_device
from helpers.experiment.experiment import evaluate_model
from helpers.datasets.settings.settings import Binned_Sets_Dataset_Settings
from helpers.datasets.datasets import Unbinned_Sets_Dataset, Binned_Sets_Dataset

results_table = Results_Table()
device = select_device()

Device:  cuda


In [ ]:
for  level in (Names_of_Levels().generator, Names_of_Levels().detector):
    for trial_range in Raw_Signal_Trial_Ranges().tuple_:
        
        Aggregated_Signal_Dataframe_Handler(
            path_to_main_datasets_dir=Paths_to_Directories().path_to_main_datasets_dir,
            level=level,
            trial_range=trial_range
        ).make_and_save(Paths_to_Directories().path_to_raw_signal_dir)

In [2]:
# from helpers.experiment.experiment import evaluate_model
# from helpers.datasets.datasets import Unbinned_Sets_Dataset

deep_sets_group = Deep_Sets_Group(
    num_sets_per_label=50,
    num_sets_per_label_sensitivity=2_000,
    q_squared_veto=Names_of_q_Squared_Vetos().loose,
    std_scale=True,
    shuffle=True,
    loss_fn=MSELoss(),
    learning_rate=3e-4, # 3e-4
    learning_rate_scheduler_reduction_factor=0.99, # 0.9
    size_of_training_batch=32,
    size_of_evaluation_batch=32,
    number_of_epochs=100, # 100
    number_of_epochs_between_checkpoints=1,
    results_table=results_table,
    device=device,
    bkg_fraction=0.5,
    bkg_charge_fraction=0.5
)

deep_sets_group.train_subset(
    levels=(Names_of_Levels().detector_and_background,),
    nums_events_per_set=(24_000,),
    remake_datasets=False
)
deep_sets_group.evaluate_subset(
    levels=(Names_of_Levels().detector_and_background,), 
    nums_events_per_set=(24_000,), 
    remake_datasets=False,
    epoch="final"
)


# evaluate_model(
#     model=deep_sets.model,
#     evaluation_dataset=Unbinned_Sets_Dataset(deep_sets.training_dataset_settings, remake=True),
#     sensitivity_evaluation_dataset=Unbinned_Sets_Dataset(deep_sets.sensitivity_evaluation_dataset_settings, remake=False),
#     results_table=results_table,
#     device=device
# )


# deep_sets_group.train_all(remake_datasets=False)
# deep_sets_group.evaluate_all(remake_datasets=True)


Loaded tensor of shape: torch.Size([2200, 24000, 4]) from ..\..\state\new_physics\data\processed\sets_unbinned_det_bkg_q2v_loose\24000_train_features.pt
Loaded tensor of shape: torch.Size([2200]) from ..\..\state\new_physics\data\processed\sets_unbinned_det_bkg_q2v_loose\24000_train_labels.pt
Loaded tensor of shape: torch.Size([2200, 24000, 4]) from ..\..\state\new_physics\data\processed\sets_unbinned_det_bkg_q2v_loose\24000_eval_features.pt
Loaded tensor of shape: torch.Size([2200]) from ..\..\state\new_physics\data\processed\sets_unbinned_det_bkg_q2v_loose\24000_eval_labels.pt

Epoch 0 complete:
    Train loss: 0.9777434163168557
    Eval loss: 0.8838631075309026

Learning rate: [0.0003]
Peak GPU memory usage:
0.78508 GB
Completed checkpoint at epoch: {ep}

Epoch 1 complete:
    Train loss: 0.8324634086811116
    Eval loss: 0.8211960046856297

Learning rate: [0.0003]
Peak GPU memory usage:
0.78508 GB
Completed checkpoint at epoch: {ep}

Epoch 2 complete:
    Train loss: 0.82630778628

c:\Users\tetha\Desktop\btokstll\logic\scripts\helpers\experiment\results_table.py:49: PerformanceWarning: indexing past lexsort depth may impact performance.
  self.table.loc[


Unloaded datasets.
Unloaded datasets.


In [ ]:
cnn_group = CNN_Group(
    num_sets_per_label=50,
    num_sets_per_label_sensitivity=2_000,
    num_bins_per_dimension=10,
    q_squared_veto=Names_of_q_Squared_Vetos().loose,
    std_scale=True,
    shuffle=True,
    loss_fn=MSELoss(),
    learning_rate=3e-4,
    learning_rate_scheduler_reduction_factor=0.9,
    size_of_training_batch=32,
    size_of_evaluation_batch=32,
    number_of_epochs=100,
    number_of_epochs_between_checkpoints=5,
    results_table=results_table,
    device=device,
    bkg_fraction=0.5,
    bkg_charge_fraction=0.5
)

# cnn_group.train_all(remake_datasets=True)
# cnn_group.evaluate_all(remake_datasets=True)
cnn_group.plot_image_examples_all(remake_datasets=False)

In [5]:
event_by_event_group = Event_by_Event_Group(
    num_evaluation_sets_per_label=50,
    num_evaluation_sets_per_label_sensitivity=2_000,
    q_squared_veto=Names_of_q_Squared_Vetos().loose,
    std_scale=True,
    shuffle=True,
    loss_fn=CrossEntropyLoss(),
    learning_rate=3e-3,
    learning_rate_scheduler_reduction_factor=0.97,
    size_of_training_batch=10_000,
    size_of_evaluation_batch=10_000,
    number_of_epochs=300,
    number_of_epochs_between_checkpoints=5,
    results_table=results_table,
    device=device
)

event_by_event_group.train_subset(levels=[Names_of_Levels.generator,], remake_datasets=True)

# event_by_event_group.train_all(remake_datasets=True)
# event_by_event_group.evaluate_all(remake_datasets=False)

event_by_event_group.evaluate_subset(
    levels=[Names_of_Levels().generator,], 
    nums_events_per_set=[24_000, 6_000, 70_000], 
    remake_datasets=True,
    epoch="final"
)

# ebe = event_by_event_group.get_individual(level=Names_of_Levels().detector)
# evaluate_model(
#     model=ebe.model,
#     evaluation_dataset=Binned_Sets_Dataset(
#         settings=Binned_Sets_Dataset_Settings(
#             level=Names_of_Levels().detector,
#             split=Names_of_Splits().train,
#             num_events_per_set=24_000,
#             num_sets_per_label=50,
#             is_sensitivity_study=False,
#             q_squared_veto=Names_of_q_Squared_Vetos().loose,
#             std_scale=True,
#             shuffle=True,
#             path_to_main_datasets_dir=Paths_to_Directories().path_to_main_datasets_dir,
#             path_to_raw_signal_dir=Paths_to_Directories().path_to_raw_signal_dir,
#             path_to_raw_bkg_dir=Paths_to_Directories().path_to_raw_bkg_dir
#         ), 
#         remake=False
#     ),
#     sensitivity_evaluation_dataset=Binned_Sets_Dataset(
#         ebe._get_sensitivity_evaluation_set_dataset_settings(num_events_per_set=24_000), 
#         remake=False
#     ),
#     results_table=results_table,
#     device=device,
#     epoch=10
# )

Making binned events dataset.
Number of NA values: 
 q_squared        0
costheta_mu      0
costheta_K       0
chi              0
dc9              0
dc9_bin_index    0
dtype: int64
Removed rows that have a NaN.
Applying standand scale.
Applied standard scale.
Shuffled dataframe.
Saved tensor of shape: torch.Size([21021610, 4]) to ..\..\state\new_physics\data\processed\events_binned_gen_q2v_loose\train_features.pt
Saved tensor of shape: torch.Size([21021610]) to ..\..\state\new_physics\data\processed\events_binned_gen_q2v_loose\train_labels.pt
Saved tensor of shape: torch.Size([44]) to ..\..\state\new_physics\data\processed\events_binned_gen_q2v_loose\train_bin_map.pt
Made binned events dataset.
Loaded tensor of shape: torch.Size([21021610, 4]) from ..\..\state\new_physics\data\processed\events_binned_gen_q2v_loose\train_features.pt
Loaded tensor of shape: torch.Size([21021610]) from ..\..\state\new_physics\data\processed\events_binned_gen_q2v_loose\train_labels.pt
Loaded tensor of shape

c:\Users\tetha\Desktop\btokstll\logic\scripts\helpers\experiment\results_table.py:49: PerformanceWarning: indexing past lexsort depth may impact performance.
  self.table.loc[


Unloaded datasets.
Unloaded datasets.
Making binned sets dataset.
Number of NA values: 
 q_squared        0
costheta_mu      0
costheta_K       0
chi              0
dc9              0
dc9_bin_index    0
dtype: int64
Removed rows that have a NaN.
Applying standand scale.
Applied standard scale.
Shuffled dataframe.
Saved tensor of shape: torch.Size([2200, 6000, 4]) to ..\..\state\new_physics\data\processed\sets_binned_gen_q2v_loose\6000_eval_features.pt
Saved tensor of shape: torch.Size([2200]) to ..\..\state\new_physics\data\processed\sets_binned_gen_q2v_loose\6000_eval_labels.pt
Saved tensor of shape: torch.Size([44]) to ..\..\state\new_physics\data\processed\sets_binned_gen_q2v_loose\6000_eval_bin_map.pt
Made binned sets dataset.
Loaded tensor of shape: torch.Size([2200, 6000, 4]) from ..\..\state\new_physics\data\processed\sets_binned_gen_q2v_loose\6000_eval_features.pt
Loaded tensor of shape: torch.Size([2200]) from ..\..\state\new_physics\data\processed\sets_binned_gen_q2v_loose\60

c:\Users\tetha\Desktop\btokstll\logic\scripts\helpers\experiment\results_table.py:49: PerformanceWarning: indexing past lexsort depth may impact performance.
  self.table.loc[


Unloaded datasets.
Unloaded datasets.
Making binned sets dataset.
Number of NA values: 
 q_squared        0
costheta_mu      0
costheta_K       0
chi              0
dc9              0
dc9_bin_index    0
dtype: int64
Removed rows that have a NaN.
Applying standand scale.
Applied standard scale.
Shuffled dataframe.
Saved tensor of shape: torch.Size([2200, 70000, 4]) to ..\..\state\new_physics\data\processed\sets_binned_gen_q2v_loose\70000_eval_features.pt
Saved tensor of shape: torch.Size([2200]) to ..\..\state\new_physics\data\processed\sets_binned_gen_q2v_loose\70000_eval_labels.pt
Saved tensor of shape: torch.Size([44]) to ..\..\state\new_physics\data\processed\sets_binned_gen_q2v_loose\70000_eval_bin_map.pt
Made binned sets dataset.
Loaded tensor of shape: torch.Size([2200, 70000, 4]) from ..\..\state\new_physics\data\processed\sets_binned_gen_q2v_loose\70000_eval_features.pt
Loaded tensor of shape: torch.Size([2200]) from ..\..\state\new_physics\data\processed\sets_binned_gen_q2v_lo

c:\Users\tetha\Desktop\btokstll\logic\scripts\helpers\experiment\results_table.py:49: PerformanceWarning: indexing past lexsort depth may impact performance.
  self.table.loc[


Unloaded datasets.
Unloaded datasets.


In [ ]:
results_table.table

In [ ]:
import pandas

pandas.read_pickle("../../state/new_physics/data/raw/bkg/mu_sideb_generic_charge_eval.pkl")